In [6]:
import numpy as np
import random
import copy
from collections import Counter
# Note: To use this import: pip install ipynb
from ipynb.fs.full.mahjong_rules import discard_tile, draw_tile, update_state, display_tiles
from ipynb.fs.full.mahjong_rules import initialize_simple_mahjong, is_winning_hand

ImportError: cannot import name 'discard_tile' from 'ipynb.fs.full.mahjong_rules' (unknown location)

### Initialize Game State

In [4]:
game_state = initialize_simple_mahjong()
display_tiles(game_state)
# state1 = copy.deepcopy(game_state)

NameError: name 'initialize_simple_mahjong' is not defined

In [64]:
# policy.N_dict
policy.N[0]

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [1]:
MonteCarloTreeSearch(game_state,3)

NameError: name 'MonteCarloTreeSearch' is not defined

In [53]:
random_seed = random.choices(np.arange(33*4),k=14)
state = np.zeros([33,4])
for i in random_seed:
    # int(i/4) gives tile no., i%4 gives copy no.
    state[int(i/4),i%4]=1
s_hash = hash(state.tobytes())
s_hash

7947700045660605086

In [71]:
class Policy(object):
    def __init__(self,n_tiles,m_simulations,player_hand_size):
        # Maximum number of moves is about n_tiles
        self.N = np.zeros([m_simulations*n_tiles,player_hand_size])
        self.Q = np.zeros([m_simulations*n_tiles,player_hand_size])
        self.state_dict = {}
        self.N_dict = {}
        self.n_tiles = n_tiles
        # Hyperparameters
        self.exploration_parameter = 1
        self.depth = 3
        self.gamma = 0.95
    
def MonteCarloTreeSearch(state,m_simulations):
    # define policy globally so that dont need to keep passing it around
    global policy
    policy = Policy(m_simulations=m_simulations,
                    n_tiles = len(state)*len(state[0]), 
                    player_hand_size=8)
    
    s_hash = hash(state.tobytes())
    policy.state_dict.update({0:state})
    policy.N_dict.update({s_hash:0})
    
    for k in range(m_simulations):
        simulate(state, policy.depth)
    
    # state = 2D array representing mahjong game state np.array([33,4])
    s_hash = hash(state.tobytes()) # s_hash = state converted to bytes (e.g., -6026512037226545482, 7947700045660605086)
    s = policy.N_dict[s_hash] # s = index (e.g., 1,2,3,..)
    
    # policy.Q[s,possible_a] is updated globally in the "simulate" function
    optimal_action = np.argmax(policy.Q[s,:])
    
    return optimal_action

def simulate(state, depth):
    if is_winning_hand(state): # NEED FUNCTION #
        return get_reward(state) # NEED FUNCTION #

    if depth <= 0:
        return get_reward(state)
      
    s_hash = hash(state.tobytes())
    if s_hash not in policy.N_dict.keys():
        new_index = max(policy.N_dict.values())+1
        policy.state_dict.update({new_index:state})
        policy.N_dict.update({s_hash:new_index})

    s = policy.N_dict[s_hash]
    
    a = explore(s)
    
    # Return a new state - discard tile and draw tile. Also return the reward for that state
    next_state, reward = update_state(state, a) # NEED FUNCTION #
    
    q = reward + policy.gamma*simulate(next_state, depth-1)
    policy.N[s,a] += 1
    policy.Q[s,a] += (q-Q[s,a])/policy.N[s,a]
    return q # q is returned to itself recursively, but not to MonteCarloTreeSearch

# np.inf*(N[s,a] == 0) returns infinity if the state has yet to be explored
# more efficient version that returns warning message: np.inf*(Nsa == 0) + np.sqrt(np.log(Ns)/Nsa)
def exploration_bonus(Nsa,Ns):
    if Nsa == 0:
        return np.inf
    else:
        return np.sqrt(np.log(Ns)/Nsa)

# equation (9.1) - Q(s,a)+c*sqrt(logN(s)/N(s,a))
def explore(s):
    Ns = np.sum(policy.N[s,:])
    policy.Q[s,:] = policy.exploration_parameter * exploration_bonus(policy.N[s,:],Ns)
    explore_action = np.argmax(policy.Q[s,:])
    return explore_action

def random_policy(state):
    while not is_winning_hand(state):
        try:
            state = choose_random_action(state)
        except IndexError:
            raise Exception("Non-terminal state has no possible actions: " + str(state))
    return get_reward(state)

In [67]:
exploration_bonus(Nsa)